In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import altair
import seaborn
RANDOM_SEED = 511
rng = np.random.default_rng(RANDOM_SEED)
seaborn.set_theme(style="darkgrid")
altair.renderers.enable('mimetype')

db_url: str = os.getenv('DB_URL')

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from lib import wrap_labels, adjust_pairplot_axes

In [ ]:
sql = """
select 
    z.zip_code,
    z.neighborhood_name,
    z.borough_name,
    CASE WHEN z.is_harlem THEN 'Harlem' 
        WHEN z.borough_name = 'Manhattan' THEN 'Manhattan (Except Harlem)'
        ELSE z.borough_name END AS is_harlem,
    coalesce(total_admissions_3_2017, 0) AS "Estimated OTP Admissions (2017)",
    coalesce(total_admissions_3_2019, 0) AS "Estimated OTP Admissions (2019)",
    coalesce(otp_capacity, 0) AS "Reported OTP Capacity (2019)",
    coalesce(avg_daily_enrollment_2019, 0) AS "Reported Daily Avg OTP Enrollment (2019)",
    coalesce(patient_admissions_3_2019, 0) AS "Estimated OTP Patient Residents (2019)",
    coalesce(harlem_patient_admissions_3_2019, 0) As "Estimated OTP Patient Residents (Harlem Programs, 2019)",
    coalesce(opioid_burden_2019, 0) AS "Opioid Burden (2019)",
    z.population_total AS "Estimated Population (2020)"
from zip_codes z
    left join zip_code_otp_analysis za on z.zip_code = za.zip_code 
"""
zip_df = pd.read_sql(sql, con=db_url)


zip_df['Estimated OTP Patient Residents (Non-Harlem Programs, 2019)'] = \
    zip_df['Estimated OTP Patient Residents (2019)'] - zip_df["Estimated OTP Patient Residents (Harlem Programs, 2019)"]
zip_df['Estimated Fraction of OTP Admissions (2019)'] = zip_df["Estimated OTP Admissions (2019)"] \
    / zip_df["Estimated OTP Admissions (2019)"].sum()
zip_df['Estimated Fraction of OTP Patient Residents (2019)'] = zip_df["Estimated OTP Patient Residents (2019)"] \
    / zip_df["Estimated OTP Patient Residents (2019)"].sum()
zip_df['Estimated Fraction of Harlem OTP Patient Residents (2019)'] =  zip_df["Estimated OTP Patient Residents (Harlem Programs, 2019)"] / \
    zip_df["Estimated OTP Patient Residents (Harlem Programs, 2019)"].sum()
zip_df['Fraction of Opioid Burden (2019)'] =  zip_df["Opioid Burden (2019)"] / \
    zip_df["Opioid Burden (2019)"].sum()

for c in [
    'zip_code',
    'neighborhood_name',
    'borough_name',
    'is_harlem'
]:
    zip_df[c] = pd.Categorical(zip_df[c])

zip_df = zip_df.set_index('zip_code')

In [ ]:
measures = [
    'Estimated OTP Admissions (2019)',
    'Reported OTP Capacity (2019)',
    'Reported Daily Avg OTP Enrollment (2019)',
    'Estimated OTP Patient Residents (2019)',
    'Opioid Burden (2019)'
]

p = seaborn.pairplot(
    zip_df[measures + ['is_harlem']], 
    hue='is_harlem',
    corner=True
)
adjust_pairplot_axes(zip_df, measures, p)

In [ ]:
def plot_xy_interactive(df, x_col, y_col, color_col, identity_line=True):
    subset = df[[x_col, y_col]]
    subset_max_pct = subset.max().max()
    domain_max = subset_max_pct * 1.05
    scale = altair.Scale(domain=[0, domain_max])

    plot = (altair.Chart(df)
        .properties(height=500, width=500)
        .mark_circle(size=60)
        .encode(
            x=altair.X(x_col, scale=scale),
            y=altair.Y(y_col, scale=scale),
            color=color_col,
            tooltip=list(df.columns)
        )
        .interactive()
    )
    if identity_line:
        line = pd.DataFrame({
            x_col: [0, domain_max],
            y_col: [0, domain_max],
        })
        line_plot = altair.Chart(line)\
            .mark_line(color='gray', strokeDash=[2, 4])\
            .encode(x=x_col, y=y_col)
        return plot + line_plot
    else:
        return plot


In [ ]:
plot_xy_interactive(
    zip_df,
    x_col = 'Estimated Fraction of OTP Patient Residents (2019)',
    y_col = 'Estimated Fraction of OTP Admissions (2019)',
    color_col='is_harlem'
)

In [ ]:

(altair.Chart(zip_df)
    .properties(height=500, width=500)
    .mark_circle(size=60)
    .encode(
        x='Opioid Burden (2019)',
        y='Reported OTP Capacity (2019)', 
        color='is_harlem',
        tooltip=list(zip_df.columns)
    )
    .interactive()
)

In [ ]:
zip_program_metrics = (
    zip_df[[
        'Estimated OTP Patient Residents (2019)',
        'Estimated OTP Admissions (2019)',
        'Reported OTP Capacity (2019)',
        'Reported Daily Avg OTP Enrollment (2019)',
        'Fraction of Opioid Burden (2019)',
        "Estimated Population (2020)"
    ]]
    .groupby(zip_df['is_harlem'])
    .sum()
)

zip_program_metrics = zip_program_metrics / zip_program_metrics.sum()
ax = zip_program_metrics.loc['Harlem'].plot(
    kind='bar',
    ylabel="Harlem's Fraction of NYC total",
    figsize=(10, 6)
)
plt.xticks(rotation=0);
wrap_labels(ax, 15)
zip_program_metrics


In [ ]:
zip_patient_metrics = (
    zip_df[[
        "Estimated OTP Patient Residents (Harlem Programs, 2019)",
    ]]
    .groupby(zip_df['is_harlem'])
    .sum()
)
zip_patient_metrics = zip_patient_metrics / zip_patient_metrics.sum()
zip_patient_metrics = zip_patient_metrics.sort_values("Estimated OTP Patient Residents (Harlem Programs, 2019)", ascending=False)
ax = (zip_patient_metrics
    .plot(
        kind='bar',
        xlabel='Patient Residence',
        ylabel='Fraction of Patients in Harlem Programs',
        legend=False,
        title='Reported Residence of Harlem OTP Patients, 2019'
    )
)
plt.xticks(rotation=0)
wrap_labels(ax, 10)
zip_patient_metrics


In [ ]:
borough_program_metrics = zip_df[[
    'Estimated OTP Admissions (2019)',
    'Reported OTP Capacity (2019)',
    'Reported Daily Avg OTP Enrollment (2019)',
    'Estimated OTP Patient Residents (2019)',
    'Opioid Burden (2019)',
    "Estimated Population (2020)"
]].groupby(zip_df['is_harlem']).sum()
borough_program_metrics = borough_program_metrics / borough_program_metrics.sum()
borough_program_metrics = borough_program_metrics.sort_values('Estimated OTP Patient Residents (2019)', ascending=False)
ax = borough_program_metrics.plot(
    kind='bar', 
    xlabel='',
    ylabel='Fraction of NYC Total'
)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
wrap_labels(ax, 10)

# Sort of easier to see pivoted
ax2 = borough_program_metrics.T.plot(
    kind='bar', 
    ylabel='Fraction of NYC Total',
    figsize=(10,6)
)
ax2.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# plt.xticks(rotation=0)
wrap_labels(ax2, 15)

borough_program_metrics